
# Additive Synthesis  
### Synthesizers: a computer science perspective - G. Tzanetakis, University of Victoria 


In this notebook we explore the basic idea of additive synthesis in which complex sounds are created by adding individual sinusoidal signals with time-varying amplitude envelopes. This allows the creation of complex sounds but requires more computational resources and careful analysis of existing sounds to obtain the partials and corresponding envelopes. 

A note on terminology. A **harmonic** refers to a peak in the magnitude spectrum at a frequency which is an integer multiple (harmonic) of a fundamental frequency. A **partial** is a more general term referring to any peak in the magnitude spectrum which in some cases might not be exactly harmonic. For an example see the bell sound at the bottom of this notebook. 


In [ ]:
import numpy as np
import IPython.display as ipd

import numpy as np
import IPython.display as ipd


plot_library = 'matplotlib'
#plot_library = 'matplotlib_xkcd'
#plot_library = 'bokeh'

if (plot_library=='bokeh'):
    import bokeh 
    from bokeh.io import output_notebook
    from bokeh.plotting import figure, output_file, show
    output_notebook()

    def plot(data_list,label_list=[],xlabel='', ylabel='', title=''):
        p = figure(plot_height=300, plot_width=600, title='Synthesizers')
        for data in data_list: 
            p.line(np.arange(0,data.shape[0]), data)
        show(p)
        
if (plot_library=='matplotlib'): 
    %matplotlib notebook 
    import matplotlib.pyplot as plt
    def plot(data_list,label_list=[],xlabel='', ylabel='', title=''):
        fig, ax = plt.subplots(figsize=(8,4))
        for (data,label) in zip(data_list, label_list): 
            plt.title('Synth-CS: '+title)
            plt.xlabel(xlabel)
            plt.ylabel(ylabel)
            plt.plot(np.arange(0, data.shape[0]), data, label=label)
        if (label_list):
            ax.legend()
        
        
if (plot_library=='matplotlib_xkcd'): 
    %matplotlib notebook 
    import matplotlib.pyplot as plt

    def plot(data_list,label_list=[],xlabel='', ylabel='', title=''):
        fig, ax = plt.subplots(figsize=(8,6))  
        plt.xkcd()
        if not(label_list):
            for d in enumerate(data_list): 
                label_list.append('')
        for (data,label) in zip(data_list, label_list): 
            plt.title('Synth-CS: '+title)
            plt.xlabel(xlabel)
            plt.ylabel(ylabel)
            plt.plot(np.arange(0, data.shape[0]), data, label=label)
            ax.legend()

Let's create a sound by compibing three sinusoid harmonics. You can see that we get a more complex time-domain waveform plot. 

In [ ]:
srate = 44100      # sampling rate 
duration = 1       # duration in seconds 
freq = 120         # frequency 

t = np.linspace(0,duration,srate*duration)     # time instances of samples 
h0 =  0.5 * np.sin(2*np.pi*freq*t+np.pi/4);    # fundamental
h1 = 0.0 *  np.sin(2*np.pi*2*freq*t);          # octave  
h2 = 0.2 * np.sin(2*np.pi*3*freq*t);           # octave+fifth 

# play the corresponding audio 
data = 0.3 * (h0 + h1 + h2) 
print(data.shape[0])
ipd.Audio(data,rate=srate)

In [ ]:
plot([data[0:1000]], ['three_harmonics'], xlabel = 'Samples', ylabel='Amplitude', title='Additive Synthesis - 3 harmonics ')


In [ ]:
import boreal 
boreal.spectrum((data, 44100))

Notice the click at the end points of the sound example. By using envelopes we can get a smoother sound with no clicks and much more complex structure. The envelope function takes as an input argument a list of tuples that describe line ramps/segments in time. This allows 
us to create complex envelopes and is a generalization of the standard 
ADSR (Attack, Decay, Sustain, Release) envelope. Each segment is characterized by a tuple: (target_value, time to target value, and time to hold value). Using this additive model we can play different notes 
of our complex sound. The next two examples are based on PureData examples from the Andy Farnell book "Designing Sound". 

In [ ]:
import numpy as np
 

def envelope(segments,srate,duration): 
    nsamples = int(srate*duration)
    value = 0.0
    segment_index = 0 
    data = np.zeros(nsamples)
    segment_sample = 0 
    prev_target = 0.0

    for i in np.arange(nsamples): 
        if (segment_index < len(segments)): 
            target = segments[segment_index][0]
            ramp_time = segments[segment_index][1]
            delay_time = segments[segment_index][2]
            
            ramp_samples = (ramp_time / 1000.0) * srate 
            delay_samples = (delay_time / 1000.0) * srate
            
            if i < segment_sample + ramp_samples: 
                incr = (target-prev_target) / ramp_samples 
            elif i < segment_sample + ramp_samples + delay_samples: 
                incr = 0.0 
            else: 
                if ramp_samples != 0.0: 
                    incr = (target-prev_target) / ramp_samples 
                else: 
                    incr = 0.0 
                segment_sample = i 
                segment_index = segment_index+1 
                prev_target = target 
            value = value + incr 
        data[i] = value
    return data  
    

s1 = [(0.8, 50, 0), (1,200,50), (0.5, 900, 250), (0,1000, 1150)]
s2 = [(0.8, 100, 0), (0.35, 200, 100), (0.2, 1200, 1200), (0,2000, 2400)]
s3 = [(0.9, 120, 0), (0.45, 500, 120), (0, 1000, 4000)]
s4 = [(0.95,400,100), (0.2, 400, 500), (0.3, 900, 900), (0, 1000, 1900)]

def sinusoid(freq=440.0, dur=1.0, srate=44100.0, amp=1.0, phase = 0.0): 
    t = np.linspace(0,dur,int(srate*dur))
    data = amp * np.sin(2*np.pi*freq *t+phase)
    return data

dur = 4.0 
f0 = 200 
penv1 = envelope(s1, srate, dur)
penv2 = envelope(s2, srate, dur)
penv3 = envelope(s3, srate, dur)
penv4 = envelope(s4, srate, dur)

osc1 = sinusoid(f0, dur=dur)
osc2 = sinusoid(2*f0, dur=dur)
osc3 = sinusoid(3*f0, dur=dur)
osc4 = sinusoid(4*f0, dur=dur)
output1 = 0.25*(penv1 * osc1 + penv2 * osc2 + penv3 * osc3 + penv4 * osc4)

f0 = 350 
osc1 = sinusoid(f0, dur=dur)
osc2 = sinusoid(2*f0, dur=dur)
osc3 = sinusoid(3*f0, dur=dur)
osc4 = sinusoid(4*f0, dur=dur)
output2 = 0.25*(penv1*osc1 + penv2 * osc2 + penv3 * osc3 + penv4 * osc4)

plot([penv1, penv2, penv3, penv4], ['Partial envelope 1', 'Partial envelope 2', 'Partial envelope 3', 'Partial envelope 4'])

data = np.concatenate([output1, output2])
ipd.Audio(data,rate=srate)

In [ ]:
boreal.render((data, srate), widgets=['time_waveform', 'spectrum', 'circulareq'])

We can create quite complex sound by introducing more partials and inharmonicity. Below you can hear the sound of bell. The frequencies 
of the partials and the corresponding envelopes can be derived by 
manual or automatic inspection of the magnitude spectrogram of a bell sound recording. 

In [ ]:

def partial(f0, multiplier, t1, t2, t3, srate, duration): 
    osc = sinusoid(f0 * multiplier, dur=duration) 
    env = envelope([(0.0, t3, 0.0), (1.0, t1, 0.0), (0.0, t2, t1)], 
                       srate, 10.0)
    return env * osc

p1 = partial(100, 1.0, 300, 9000, 400, srate, 10)
p2 = partial(100, 2.01, 700, 8000, 100, srate, 10)
p3 = partial(100, 4.02, 400, 7000, 32, srate, 10)
p4 = partial(100, 6.06, 200, 6000, 20, srate, 10)
p5 = partial(100, 8.12, 37, 5000, 17, srate, 10)
p6 = partial(100, 10.25, 31, 4000, 12, srate, 10)
p7 = partial(100, 12.5, 20, 3000, 7, srate, 10)
p8 = partial(100, 14.8, 15, 1500,5, srate, 10)
p9 = partial(100, 17, 5, 100, 2, srate, 10)
p10 = partial(100, 19.02, 2, 500, 0, srate, 10)

xstart = 5000 
xend = 22000 
plot([p3[xstart:xend], p2[xstart:xend], p1[xstart:xend]], ['p3','p2','p1'], title='Partial evolution')


signal = (p1+p2+p3+p4+p5+p6+p7+p8+p9+p10)*0.01 
plot([signal], ['Bell'], title='Bell')
ipd.Audio(signal, rate=srate)

In [ ]:
boreal.spectrum((signal,srate))